# Automatic data analysis and plotting.

### __How to use__
- Put the files to be analyzed in the Data folder.
- Click Run All button. 
- The processed files will be put in the Output folder. 
- The exported graph will be placed in Graph export folder. 


### __Supported experiment type__
 - __Single experiment__
    - With/ without LC data.
    - With / without temperature data
    - With / without pressure data
    - Flow data (with automatic removing of GC injection data points, this help reducing overshooting of FE due to inconsistent flow.)
        - Recorded with DryCal software. 
        - Recorded with the custom Python script.
        
- __Multiplexing from 2 units and more__
    - With / without temperature data
    - With / without pressure data
    - LC data is supported at this version (provided the LC file is placed directly inside eash unit's folder)
    - Automatic flow data slicing and removing GC valve switching and GC injection data points. The flow data has to be recorded using the custom python script. 
    - Compatible with 1 and more GC folders / flow meter files for one multiplex experiment.

### __Files requirement__
- **** The naming of the files have to be strictly follow the following convention to ensure the smooth running of the script.
- Unless specified otherwise, no space can be used. Please use underscore (_).
- Unless specified otherwise (LC data has different format), the date info has to be in the format of yyyymmdd. For example: 20230622.

- __Single experiment__

    - __Pressure data (optional):__ Two files are required.
        - pressure file: 
            - In the format of yyyymmdd_XXX-pressure.csv 
            - Inside the file, ensure the columns name are UXLiquid(Read)[mbar] and UXGas(Read)[mbar] (X is the number of the unit)
        - pressure log:
            - In the format of yyyymmdd_XXX-pressurelog.txt
    - __Temperature data (optional):__ One file is required.
        - temperature file:
            - In the format of yyyymmdd_XXX-temperature.csv 
            - The script will take column B as the cell temperature and column D as a room temperature. 
    - __Flow data (required):__ Two files are required. 
        - Flow file: 
            - In the format of yyyymmdd_XXX-flow.csv 
            - This can be from the custom script or from DryCal program. 
        - GC files folder:
            - In the format of yyyymmdd_XXX-GC.zip
            - This is used to remove the artifact points resulted from GC injection. We found that this usually lead to incorrect FE calculation. By default, the script will remove all flow data recorded within the time windows of 75 seconds before and after the GC injection.

- __Multiplex experiment__
    - Capable of analyzing  multiplex experiment from 2 cells onward.
    - For a folder containing a multiplex experiment, the name of the folder must be started with 'Multiplex'
    - The script will take anything between the first and second (if any) underscore as the experiment name: eg. Multiplex2_30p-flow-NoEDC, the exp name will be 30p-flow-NoEDC. Multiplex_20230805_30p-flow-NoEDC, exp name will be 20230805.

    -__Flow data (required):__
        - Must be in the format of collection-[sequence of unit]-flow.csv. For example, if the the said flow data stores the flow data from unit U2 and U4 (alternating) the naming would be collection-U2-U4-flow.csv. 

    -__GC data (required):__
        - Must be in the format of collection-[sequence of unit]-GC.zip. For example, if the the said flow data stores the flow data from unit U2 and U4 (alternating) the naming would be collection-U2-U4-GC.zip 
    
    -__Echem files (required):__ 
        - Echem files for each unit must be stored inside a unit folder (naming UX, where X is the unit number.)

    -__Pressure data (optional):__ 
        - pressure file: 
            - In the format of yyyymmdd_XXX-pressure.csv 
            - Inside the file, ensure the columns name are UXLiquid(Read)[mbar] and UXGas(Read)[mbar] (X is the number of the unit). The script will slice take the pressure from the number in this file. Make sure there are columns from U1 to U8. 
        - pressure log:
            - In the format of yyyymmdd_XXX-pressurelog.txt

    -__Temperature data (optional):__ One file is required.
        - temperature file:
            - In the format of yyyymmdd_XXX-temperature.csv 
            - The temp column of each unit is in the format of Unit X Last (C). Note that here the space is used. The room temperature is in the column called RT-external Last (C). 

In [ ]:
import warnings
from ryan.analyzer import autoanalyze as aa
from ryan.analyzer import autoplot as ap
from ryan.analyzer import auxiliary as aux

## Data slicing
- Clean 'Recipe/data_for_dgbowl' for any old data and prepare for the current data.
- Slice data and prepare them for processing. 

In [ ]:
aux.clear_dir('Recipe/data_for_dgbowl')
aux.data_slicer()

## yadg and dgpost
- ### Multiprocessing
    - __False:__
        - The script will process the data folder by folder using only one CPU core. This method can be slower in many cases but will ensure that the sure can use a computer while running the script. 
    - __True:__
        - The script will use ALL of the CPU cores avilable in the machine to process the data. Each core will responsible for one folder. 
        - This can greatly reduce the time required for processing the data, and the processing time will be greatly be affected by the computational power of the machine. 
        - WARNING!! Using this feature can make the machine become lag to the point where other task cannot be run and cause programs to crash. It is highly adviced to leave the machine running the script and not working on other programs. In case of large files (especially for multiplex8 experiment), the system can run out of memory (RAM), resulting in a slow down of the system. In such case, single core processing is adviced (Multiprocessing=False). 

- ### Mannual recipe
    - This feature allows the user to choose whether they will manually create the recipe for yadg and dgpost themselves or will let the system to dynamically create the recipes based on the data avilability and the experiment being run. 
    - __manual_recipe=True__: The user will create a custom recipe for yadg/dgpost and put the recipes in Recipe/yadg and Recipe/dgpost
    - __manual_recipe=False__: The user will let the system to dynamically generate the recipe for yadg / dgpost.
        - Currently,only the dgpost recipe will be adjusted. Here are the following points that the script will look into and adjust the recipe accordingly:
            - __Temperature__:
                - If there is a temperature file, the recipe will read the temp. data
                - If there is no temperature file, the recipe will put the default of 20 Celcius. 
            - __Pressure__:
                - The recipe will extract the pressure file only if there is a pressure files. 
            - __pH__:
                - The recipe will extract the pH from the metadata file.
            - __LC data__:
                - The script will generate the recipe for LC files, only if there is LC data. 
            - __Correctly assgined the charge to C for each experiment__:
                - The charge for C wil be assigned to 4 for CO<sub>2</sub>RR and 2 for CORR for the caluclation of FE for GC. 
                - The information on the reaction being performed will be extracted from the metadata file. 

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl") #Suppress UserWarning from openpyxl library
aa.auto_dgbowl(multi_processing=False, show_log=False, manual_recipe=False, clean_temp_dir=True)


## Graph plotting

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl") #Suppress UserWarning from openpyxl library
ap.dir_make_graph(type='GC', format='png', save_fig=False, path_output='Graph_Export')
ap.dir_make_graph('Output',type='LC', save_fig=False, path_output='Graph_Export')